<a href="https://colab.research.google.com/github/ZaFuh/deepsort/blob/main/Sahi_Deepsort_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!unzip Trial5.zip


Archive:  Trial5.zip
   creating: Trial5/
  inflating: Trial5/best.pt          
   creating: Trial5/deep_sort/
   creating: Trial5/deep_sort/.git/
   creating: Trial5/deep_sort/.git/branches/
  inflating: Trial5/deep_sort/.git/config  
  inflating: Trial5/deep_sort/.git/description  
  inflating: Trial5/deep_sort/.git/HEAD  
   creating: Trial5/deep_sort/.git/hooks/
  inflating: Trial5/deep_sort/.git/hooks/applypatch-msg.sample  
  inflating: Trial5/deep_sort/.git/hooks/commit-msg.sample  
  inflating: Trial5/deep_sort/.git/hooks/fsmonitor-watchman.sample  
  inflating: Trial5/deep_sort/.git/hooks/post-update.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-applypatch.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-commit.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-merge-commit.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-push.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-rebase.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-receive.sample  

In [11]:
cd Trial5


/content/Trial5


In [12]:
ls

best.pt     main.py      requirements.txt  vid1.mp4.mp4  vid3.mp4.mp4
deep_sort/  model_data/  tracker.py        vid2.mp4.mp4


In [13]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.1 MB/s eta 0:00:00


In [14]:
!pip -q install ultralytics sahi==0.11.15 lap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 10.0 MB/s eta 0:00:00


In [17]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [15]:
#along with all previous steps from 4.py, install SAHI: !pip -q install ultralytics sahi==0.11.15 lap

import os
import cv2
import torch
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel

# allowlist DetectionModel for torch weights-only unpickler (safe if you trust weights)
torch.serialization.add_safe_globals([DetectionModel])

# DeepSORT imports
from deep_sort.deep_sort.tracker import Tracker as DeepSort
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection

# SAHI imports
from sahi.models.yolov8 import Yolov8DetectionModel
from sahi.predict import get_sliced_prediction

def xyxy_to_xywh(box):
    """Convert [x1,y1,x2,y2] box to [cx,cy,w,h]."""
    x1, y1, x2, y2 = box
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2
    cy = y1 + h / 2
    return [cx, cy, w, h]

# Paths
VIDEO_IN = "vid1.mp4"
VIDEO_OUT = "out_tracked_sahi.mp4"
WEIGHTS = "best.pt"  # your YOLOv11 weights

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# (Optional) Load YOLO just to access class names if you want later
ultra_model = YOLO(WEIGHTS)
ultra_model.to(device)
names = ultra_model.model.names if hasattr(ultra_model, "model") else {}

# SAHI detection model (works with Ultralytics v8/v11 under the hood)
sahi_model = Yolov8DetectionModel(
    model_path=WEIGHTS,
    confidence_threshold=0.25,  # tune as needed
    device=device,
)

# Init DeepSORT
max_cosine_distance = 0.4
nn_budget = None
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = DeepSort(metric)

# Open video
cap = cv2.VideoCapture(VIDEO_IN)
if not cap.isOpened():
    raise RuntimeError(f"Could not open input video {VIDEO_IN}")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

# Safer mp4 codec
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))

# SAHI slice settings (good starting point for 1080p+; tune for your footage)
SLICE_W = 640
SLICE_H = 640
OVERLAP_W = 0.2
OVERLAP_H = 0.2
# If objects are *really* tiny, increase overlap (e.g., 0.3–0.4) or reduce slice size to 512

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # ---- SAHI sliced inference on this frame ----
    # Note: pass the frame (BGR numpy array) directly
    sahi_result = get_sliced_prediction(
        image=frame,
        detection_model=sahi_model,
        slice_height=SLICE_H,
        slice_width=SLICE_W,
        overlap_height_ratio=OVERLAP_H,
        overlap_width_ratio=OVERLAP_W,
        perform_standard_pred=False,         # only sliced pred (faster when you only want slices)
        postprocess_match_metric="IOU",      # merge overlapping slice preds
        postprocess_match_threshold=0.5,     # tune if you see duplicate boxes
    )

    # Convert SAHI predictions -> DeepSORT detections
    detections = []
    for obj in sahi_result.object_prediction_list:
        # bbox in xyxy
        x1, y1, x2, y2 = obj.bbox.to_xyxy()
        conf = float(obj.score.value)
        # SAHI category.id may be None depending on model metadata; default to 0 if missing
        cls_id = int(obj.category.id) if obj.category and obj.category.id is not None else 0

        bbox_xywh = xyxy_to_xywh([x1, y1, x2, y2])
        detections.append(Detection(bbox_xywh, conf, cls_id))

    # Update tracker with detections
    tracks = tracker.update(detections)

    # Draw tracks (ID only; add class name if you want)
    if tracks is not None:
        for track in tracks:
            bbox = track.to_tlbr()
            x1, y1, x2, y2 = map(int, bbox)
            track_id = int(track.track_id)

            # Optional: show class label (requires mapping track->last detection class if you store it)
            # Here we just draw the ID for simplicity
            color = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID:{track_id}", (x1, y1 - 6),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Ensure frame size matches writer and save
    frame_resized = cv2.resize(frame, (w, h))
    writer.write(frame_resized)

cap.release()
writer.release()
print("✅ Saved video:", VIDEO_OUT)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import